# Шаг 1: Загрузка APK с Fossdroid (LVDAndro)

Реализация взята из скриптов датасета LVDAndro: https://github.com/softwaresec-labs/LVDAndro

In [1]:
import requests
from bs4 import BeautifulSoup
import subprocess
import os
import shutil

In [2]:
DOWNLOAD_BASE_PATH = "D:/ETU/apk_all"

In [3]:
def get_menu_links():
    url_to_call = "https://fossdroid.com/"

    response = requests.get(url_to_call, headers={'User-Agent': 'Mozilla/5.0'})
    response_code = response.status_code

    if response_code != 200:
        print("Some Error")
        return

    html_content = response.content
    dom = BeautifulSoup(html_content, 'html.parser')
    all_menu_content = dom.select("a.mdl-navigation__link")

    menu_links = []

    for each_menu_content in all_menu_content:
        href_link = each_menu_content.attrs["href"]

        next_link = url_to_call+href_link[1:]+"whats_new.html"

        menu_links.append(next_link)

    return menu_links

In [4]:
def get_view_app_links(all_menu_links):
    page_links = []
    for each_menu_link in all_menu_links:
        response = requests.get(each_menu_link, headers={'User-Agent': 'Mozilla/5.0'})
        response_code = response.status_code

        if response_code != 200:
            print("Some Error")
            return

        html_content = response.content
        dom = BeautifulSoup(html_content, 'html.parser')
        all_view_app_content = dom.select("div.mdl-card__title h5")

        for each_view_app_content in all_view_app_content:

            href_link = each_view_app_content.a.attrs["href"]
            page_link = "https://fossdroid.com" + href_link

            page_links.append(page_link)

    return page_links

In [5]:
def download_one_by_one_apk_git(all_links,visiting_links_to,visiting_links_from):      
        
    if(len(all_links) >= visiting_links_from):
        
        if(len(all_links)<visiting_links_to):
            visiting_links_to = len(all_links)
            
        for i in range(visiting_links_from,visiting_links_to):
            try:
                os.chdir(DOWNLOAD_BASE_PATH) 
                response = requests.get(all_links[i], headers={'User-Agent': 'Mozilla/5.0'})
                response_code = response.status_code
                if response_code != 200:
                    print("Some Error")
                    return


                html_content = response.content
                dom = BeautifulSoup(html_content, 'html.parser')
                each_page_contents_apk = dom.select("#fd-download-app")        

                apk_name ="" 
                is_already_downloaded = False
                for each_page_content_apk in each_page_contents_apk:
                    href_link = each_page_content_apk.attrs["href"]
                    apk_link = "https://fossdroid.com"+href_link                  

                    r = requests.get(apk_link, allow_redirects=True)
                    filename = apk_link[24:]
                    apk_name = filename[:len(filename)-4]
                    print(apk_name)                
                    if (os.path.isdir(apk_name)!=True):     
                        is_already_downloaded = False
                        #os.mkdir(apk_name)
                        #os.chdir(DOWNLOAD_BASE_PATH+"/"+apk_name)
                        os.chdir(DOWNLOAD_BASE_PATH+"/")
                        open(filename, 'wb').write(r.content)
                        
                    else:
                        is_already_downloaded = True                     
                
#                 if is_already_downloaded == False:
#                     each_page_contents_git = dom.select("#fd-source-app")   
#                     for each_page_content_git in each_page_contents_git:
#                         github_link = each_page_content_git.attrs["href"]
#                         subprocess.call("git clone "+github_link+".git")
                    
            except Exception as e:
                    print(e)
                    continue 
                

In [6]:
def delete_git_unavailable_folders():
    git_unavailable_list = []
    folders = os.listdir(DOWNLOAD_BASE_PATH)
    print("Number of all downloads : ", len(folders))

    for folder in folders:
        try:
            files  = os.listdir(DOWNLOAD_BASE_PATH+"/"+folder)
            if(len(files)<2):
                git_unavailable_list.append(folder)
        except:
            continue    

    for non_git_folder in git_unavailable_list:
        try:
            shutil.rmtree(DOWNLOAD_BASE_PATH+"/"+non_git_folder)
        except:
            continue
            
    print ("Number of finalised Downloads : ", len(folders) - len(git_unavailable_list))

In [7]:
def execute_process():        
    print("****************Process Started*****************")
    
    print("****************Started Scraping****************")

    print("***********Retrieveing all menu links***********")
    all_menu_links = get_menu_links()
    
    menu_count = len(all_menu_links)
    print("Total number of menu :",menu_count)
    
    for menu_no in range(0, menu_count):             
    #for menu_no in range(8, 9): // enable to customize the search
        try:         
            print("**************Retrieveing all links**************")
            all_links = get_view_app_links(all_menu_links[menu_no:menu_no+1])
            all_links_count = len(all_links)
            print("Total number of links :",all_links_count)
                    
            for visiting_links_from in range (0, all_links_count, 20):    
            #for visiting_links_from in range (80, 100, 20): // enable to customize the search
                try:
                    visiting_links_to = (visiting_links_from+20)
                    print("Menu no = ",menu_no, " Visiting links_from = ",visiting_links_from, " Vsiting links_to = ",visiting_links_to)   
                    print("****************Downloading files****************")
                    download_one_by_one_apk_git(all_links,visiting_links_to,visiting_links_from)
                except Exception as e:
                    print(e)
                    continue 
                        
        except Exception as ex:
            print(ex)
            continue     
    
    print("***********Cleaning GitHub repositories unavaialble downloads***********")
    
    delete_git_unavailable_folders()
                
    print("****************Process Completed****************")    

In [ ]:
execute_process()